In [6]:
def SMT(model, inDistTrain_loader, inDistValid_loader, outDistValid_loader):
    # No inference is needed. Prediction  

    # In Distribution data values    
    ################################################################################    
    device = torch.device("cuda")
    model.to(device)
    softmax_indist = []
    inDistValid_y = []
    test_loss = 0
    accuracy = 0
    model.eval()
    print("passed model eval")
    with torch.no_grad():
        for inputs, labels in inDistValid_loader:
            inputs, labels = inputs.to(device),labels.to(device)
            labels_np = labels.cpu().detach().numpy()
            if len(inDistValid_y) == 0:
                inDistValid_y = labels_np
            else:
                inDistValid_y = np.concatenate((inDistValid_y,labels_np), axis = 0)   
            
            logps = model.forward(inputs)
            sf= torch.nn.Softmax(dim=1)
            softmax = sf(logps)
            softmax = softmax.cpu().detach().numpy()
            if len(softmax_indist) == 0:
                softmax_indist  =  softmax
            else:
                softmax_indist  =  np.concatenate((softmax_indist,softmax), axis = 0)  
        
        
    # Out distribution data values     
    ################################################################################
    
    device = torch.device("cuda")
    model.to(device)
    softmax_outdist = []
    outDistValid_y = []
    test_loss = 0
    accuracy = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in outDistValid_loader:
            inputs, labels = inputs.to(device),labels.to(device)
            labels_np = labels.cpu().detach().numpy()
            if len(outDistValid_y) == 0:
                outDistValid_y = labels_np
            else:
                outDistValid_y = np.concatenate((outDistValid_y,labels_np), axis = 0)   
            
            logps = model.forward(inputs)
            sf= torch.nn.Softmax(dim=1)
            softmax = sf(logps)
            softmax = softmax.cpu().detach().numpy()
            if len(softmax_outdist) == 0:
                softmax_outdist  =  softmax
            else:
                softmax_outdist  =  np.concatenate((softmax_outdist,softmax), axis = 0)        
                
    # In Distribution data values    
    ################################################################################    
    def entropy_from_distribution(p, axis):
    #return np.log(10.) + np.sum(p * np.log(np.abs(p) + 1e-11), axis=1, keepdims=True)
        return np.sum(p * np.log(np.abs(p) + 1e-11), axis=1, keepdims=True)

    def print_curve_info(safe, risky, inverse=False):
        labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
        if inverse:
            labels[safe.shape[0]:] += 1
        else:
            labels[:safe.shape[0]] += 1
        examples = np.squeeze(np.vstack((safe, risky)))
        print('AUPR (%):', round(100*sk.average_precision_score(labels, examples), 2))
        print('AUROC (%):', round(100*sk.roc_auc_score(labels, examples), 2))

        
    def entropy_stats(softmax):
        s_prob = np.amax(softmax, axis=1, keepdims=True)

        kl_all = entropy_from_distribution(softmax, axis=1)
        mean_all, var_all = np.mean(kl_all), np.var(kl_all)
        return s_prob, kl_all, mean_all, var_all
    

    (s_prob_in, kl_in, mean_in, var_in) = entropy_stats(softmax_indist)
    (s_prob_out, kl_out, mean_out, var_out) = entropy_stats(softmax_outdist)
    print_curve_info(-s_prob_in, -s_prob_out, True)
    return 1;
                

In [2]:
def MAH(model, inDistTrain, inDistValid, outDistValid):
    return 2


In [3]:
def EBM(model, inDistTrain, inDistValid, outDistValid):
    return 3;